<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/Meta_Pseudo_Labels_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Meta Pseudo Labels mar 2021 https://arxiv.org/pdf/2003.10580v4.pdf
# https://github.com/kekmodel/MPL-pytorch

In [ ]:
# @title download
# # # google images unlabeled
# !gdown 1ncx2DJ-GXqrQd6nL5UEmj6GLT4w-9qYs -O house.zip
# !unzip /content/house.zip -d /
# !rm -R /content/house/.ipynb_checkpoints

# # # 70k+gmap
# !gdown 1-CZp7TbhJLeRQpbKQCyT8ofGg89Yt137 -O gsv.zip
# !unzip /content/gsv.zip -d /
# !rm -R /content/gsv70kg/.ipynb_checkpoints
# !rm -R /content/gsv70kg/01/.ipynb_checkpoints
# !rm -R /content/gsv70kg/02/.ipynb_checkpoints
# !rm -R /content/gsv70kg/03/.ipynb_checkpoints
# !rm -R /content/gsv70kg/04/.ipynb_checkpoints
# !rm -R /content/gsv70kg/05/.ipynb_checkpoints
# !rm -R /content/gsv70kg/06/.ipynb_checkpoints


In [1]:
# @title data
import torch
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

labeled_dir='/content/gsv70kg'

labeled_data = datasets.ImageFolder(labeled_dir, transform=transform)
torch.manual_seed(0)
train_data, test_data = torch.utils.data.random_split(labeled_data, [.9,.1])
# train_data, _ = torch.utils.data.random_split(train_data, [.01,.99])
# test_data, _ = torch.utils.data.random_split(test_data, [.01,.99])
finetune_dataset = train_data

unlabel_dir='/content/house'
# unlabel_data = datasets.ImageFolder(unlabel_dir, transform=transform)

import os
from PIL import Image
class Datasetme(torch.utils.data.Dataset):
    def __init__(self, dir, transform=None):
        self.dir = dir
        self.data = os.listdir(dir)
        self.transform = transform
    def __getitem__(self, index):
        img_file = self.data[index]
        img_file = os.path.join(self.dir, img_file)
        image = Image.open(img_file).convert("RGB")
        if self.transform: image = self.transform(image)
        return image
    def __len__(self): return len(self.data)

unlabel_data = Datasetme(unlabel_dir, transform=transform)
# unlabel_data, _ = torch.utils.data.random_split(unlabel_data, [.01,.99])


batch_size = 64 # 16 is max for res152; default 64/ mainargs128
grad_acc = 1

# res152 batch16 gradacc4

num_batches=int(np.ceil(len(train_data)/batch_size))

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=4, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, pin_memory=True)

train_sampler = RandomSampler
# labeled_loader = DataLoader(labeled_data, sampler=train_sampler(labeled_data), batch_size=batch_size, num_workers=4, drop_last=True)
# unlabeled_loader = DataLoader(unlabel_data, sampler=train_sampler(unlabel_data), batch_size=batch_size * 7, num_workers=4, drop_last=True) # mu=7 ,coefficient of unlabeled batch size
# test_loader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=batch_size, num_workers=4)

labeled_loader = DataLoader(labeled_data, sampler=train_sampler(labeled_data), batch_size=batch_size*grad_acc, drop_last=True)
mu=7 # coefficient of unlabeled batch size
unlabeled_loader = DataLoader(unlabel_data, sampler=train_sampler(unlabel_data), batch_size=batch_size*grad_acc * mu, drop_last=True)
test_loader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=batch_size*grad_acc)

del labeled_data, train_data, test_data, unlabel_data


In [2]:
# @title torch augment
# https://github.com/facebookresearch/vicreg/blob/main/augmentations.py
import torch
import torchvision.transforms as transforms

class TrainTransform(object):
    def __init__(self):
        # self.transform = transforms.RandomApply([transforms.Compose([
        self.transform = transforms.Compose([
                transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # me
                transforms.RandomResizedCrop((400,640), scale=(0.7, 1.0), ratio=(0.8, 1.25), interpolation=transforms.InterpolationMode.BICUBIC),
                # transforms.RandomResizedCrop((32,32), scale=(0.7, 1.0), ratio=(0.8, 1.25), interpolation=transforms.InterpolationMode.BICUBIC),
                transforms.RandomHorizontalFlip(p=0.5), # 0.5
                transforms.Lambda(lambda x : torch.clamp(x, 0., 1.)), # clamp else ColorJitter will return nan https://discuss.pytorch.org/t/input-is-nan-after-transformation/125455/6
                transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8,), # brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8
                transforms.RandomGrayscale(p=0.2), # 0.2
                # # transforms.RandomChoice(transforms.ColorJitter , transforms.RandomGrayscale(p=1.)
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),], p=1.0),
                # # transforms.RandomSolarize(threshold=130, p=0.5)
                transforms.RandomErasing(p=1., scale=(0.1, 0.11), ratio=(1,1), value=(0.485, 0.456, 0.406)),
                # transforms.ToTensor(), # ToTensored at dataset level, no need to ToTensor again
                # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalised at dataset level. default 0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225
                ])
            # ], p=1.)

    def __call__(self, sample):
        dims = len(sample.shape)
        if dims==3: x1 = self.transform(sample) # same transforms per minibatch
        elif dims==4: x1 = transforms.Lambda(lambda x: torch.stack([self.transform(x_) for x_ in x]))(sample) # diff transforms per img in minibatch
        # x1 = self.transform(sample)
        return x1

trs=TrainTransform()


In [3]:
# @title utils
# https://github.com/kekmodel/MPL-pytorch/blob/main/utils.py
import torch
from torch import nn
from torch.nn import functional as F

def create_loss_fn():
    label_smoothing = 0 # default 0 / mainargs 0.15
    # if label_smoothing > 0:
    #     criterion = SmoothCrossEntropyV2(alpha=label_smoothing)
    # else:
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    return criterion.to(device)

from collections import OrderedDict
def module_load_state_dict(model, state_dict):
    try:
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]  # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)
    except:
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = f'module.{k}'  # add `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

def model_load_state_dict(model, state_dict):
    try: model.load_state_dict(state_dict)
    except: module_load_state_dict(model, state_dict)


def accuracy(output, target, topk=(1,)):
    output = output.to(torch.device('cpu'))
    target = target.to(torch.device('cpu'))
    maxk = max(topk)
    batch_size = target.shape[0]
    _, idx = output.sort(dim=1, descending=True)
    pred = idx.narrow(1, 0, maxk).t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(dim=0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class SmoothCrossEntropy(nn.Module):
    def __init__(self, alpha=0.1):
        super(SmoothCrossEntropy, self).__init__()
        self.alpha = alpha

    def forward(self, logits, labels):
        if self.alpha == 0:
            loss = F.cross_entropy(logits, labels)
        else:
            num_classes = logits.shape[-1]
            alpha_div_k = self.alpha / num_classes
            target_probs = F.one_hot(labels, num_classes=num_classes).float() * (1. - self.alpha) + alpha_div_k
            loss = (-(target_probs * torch.log_softmax(logits, dim=-1)).sum(dim=-1)).mean()
        return loss

class SmoothCrossEntropyV2(nn.Module):
    """NLL loss with label smoothing."""
    def __init__(self, label_smoothing=0.1):
        super().__init__()
        assert label_smoothing < 1.0
        self.smoothing = label_smoothing
        self.confidence = 1. - label_smoothing

    def forward(self, x, target):
        if self.smoothing == 0:
            loss = F.cross_entropy(x, target)
        else:
            logprobs = F.log_softmax(x, dim=-1)
            nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
            nll_loss = nll_loss.squeeze(1)
            smooth_loss = -logprobs.mean(dim=-1)
            loss = (self.confidence * nll_loss + self.smoothing * smooth_loss).mean()
        return loss


# from main
import math
from torch.optim.lr_scheduler import LambdaLR
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_wait_steps=0, num_cycles=0.5, last_epoch=-1):
    def lr_lambda(current_step):
        if current_step < num_wait_steps:
            return 0.0
        if current_step < num_warmup_steps + num_wait_steps:
            return float(current_step) / float(max(1, num_warmup_steps + num_wait_steps))
        progress = float(current_step - num_warmup_steps - num_wait_steps) / float(max(1, num_training_steps - num_warmup_steps - num_wait_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))
    return LambdaLR(optimizer, lr_lambda, last_epoch)



In [4]:
# @title ModelEMA
# expopnential moving average, smoothen model parameters
# https://github.com/kekmodel/MPL-pytorch/blob/main/models.py
import torch
import torch.nn as nn
from copy import deepcopy

class ModelEMA(nn.Module):
    def __init__(self, model, decay=0.9999, device=None):
        super().__init__()
        self.module = deepcopy(model)
        self.module.eval()
        self.decay = decay
        self.device = device
        if self.device is not None:
            self.module.to(device=device)

    def forward(self, input):
        return self.module(input)

    def _update(self, model, update_fn):
        with torch.no_grad():
            for ema_v, model_v in zip(self.module.parameters(), model.parameters()):
                if self.device is not None:
                    model_v = model_v.to(device=self.device)
                ema_v.copy_(update_fn(ema_v, model_v))
            for ema_v, model_v in zip(self.module.buffers(), model.buffers()):
                if self.device is not None:
                    model_v = model_v.to(device=self.device)
                ema_v.copy_(model_v)

    def update_parameters(self, model):
        self._update(model, update_fn=lambda e, m: self.decay * e + (1. - self.decay) * m)

    def state_dict(self):
        return self.module.state_dict()

    def load_state_dict(self, state_dict):
        self.module.load_state_dict(state_dict)



In [5]:
# @title big teacher
import torch
import torch.nn as nn
from torchvision import models
# !gdown 1ysJfdsvwMiWbCdkvFHwNqAUnJTtm6KbT -O res152adamw71.pth # ty
# !gdown 1VaPxGoaLjmt7K9VHi0FWbJ5efEZTLhwd -O res18teacher.pth # A
# !pip install bitsandbytes

model = models.resnet18(weights='DEFAULT') # 18 34 50 101 152
num_ftrs = model.fc.in_features
model.fc = nn.Sequential( # og (fc): Linear(in_features=2048, out_features=1000, bias=True)
    # nn.Linear(num_ftrs, num_classes, bias=False),
    # nn.Softmax(dim=1),
    )

# model.load_state_dict(torch.load('/content/bigTeacher.pth'))
# _, modelsd, _,_ = torch.load('/content/bigTeacher.pth').values()
# _, modelsd, _,_ = torch.load('/content/res152adamw71.pth').values()
_, modelsd, _,_ = torch.load('/content/res18teacher.pth').values()
model.load_state_dict(modelsd, strict=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# model = torch.compile(model.to(device)) # compiling teacher leads to significantly higher vram usage


model.eval()
print('uh')


# torch.save(model.state_dict(), '/content/model.pth')
# modelsd = torch.load('/content/model.pth')
# model.load_state_dict(modelsd, strict=False)
# # model = torch.compile(model.to(device))


uh


In [6]:
# @title model

num_classes = 6
# if dataset == "cifar10": depth, widen_factor = 28, 2
# elif dataset == 'cifar100': depth, widen_factor = 28, 8
# teacher_model = WideResNet(num_classes=num_classes, depth=depth, widen_factor=widen_factor, dropout=0, dense_dropout=0.2)
# student_model = WideResNet(num_classes=num_classes, depth=depth, widen_factor=widen_factor, dropout=0, dense_dropout=0.2)
import torch
import torch.nn as nn
from torchvision import models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_resnet():
    model = models.resnet18(weights='DEFAULT') # 18 34 50 101 152
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential( # og (fc): Linear(in_features=2048, out_features=1000, bias=True)
        nn.Linear(num_ftrs, num_classes, bias=False),
        nn.Softmax(dim=1),
        )
    # model = model.to(device)
    model = torch.compile(model.to(device))
    return model

class Small(nn.Module):
    def __init__(self, embed_dim, output_dim):
        super(Small, self).__init__()
        hidden_size=512
        self.lin = nn.Sequential(
            nn.Linear(embed_dim, hidden_size), nn.ReLU(),
            nn.Linear(hidden_size, hidden_size), nn.ReLU(),
            nn.Linear(hidden_size, hidden_size), nn.ReLU(),
            nn.Linear(hidden_size, hidden_size), nn.ReLU(),
            nn.Linear(hidden_size, output_dim, bias=False),
            nn.Softmax(dim=1),
        )
    def forward(self, x):
        logits = self.lin(x)
        return logits




# @title ensemble
import torch
import torch.nn as nn

class Ensemble(nn.Module):
    def __init__(self, embed_dim, output_dim):
        super().__init__()
        self.output_dim = output_dim # 6
        self.embed_dim = embed_dim
        h_dim = 512
        self.fwd = nn.Sequential(
            nn.Linear(self.embed_dim, h_dim), nn.ReLU(),
            # nn.Linear(h_dim, h_dim), nn.ReLU(),
            # Block(h_dim, h_dim, 0.5),
            Block(h_dim, h_dim),
            Block(h_dim, h_dim),
            Block(h_dim, h_dim),
            nn.Linear(h_dim, self.output_dim, bias=False),
            nn.Softmax(dim=1),
            )
    def forward(self, x):
        out = self.fwd(x)
        return out

class Block(nn.Module):
    def __init__(self, in_dim, out_dim, drop=None):
        super().__init__()
        if drop: self.fwd = nn.Sequential(nn.BatchNorm1d(in_dim), nn.Dropout(drop), nn.Linear(in_dim, out_dim), nn.ReLU(),)
        else: self.fwd = nn.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU(),)
    def forward(self, x):
        return x + self.fwd(x)

# teacher_model = Ensemble(2048, 6).to(device)
# teacher_model = torch.compile(Ensemble(2048, 6).to(device))




# teacher_model = Small(num_ftrs,6).to(device)
teacher_model = torch.compile(Small(num_ftrs,6).to(device))
# teacher_model = get_resnet()






student_model = get_resnet()
_, modelsd, _,_ = torch.load('/content/res18teacher.pth').values()
student_model.load_state_dict(modelsd, strict=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student_model = student_model.to(device)
# student_model = torch.compile(student_model.to(device)) #


avg_student_model = None
ema = 0.995 # default 0 / mainargs 0.995

if ema > 0: avg_student_model = ModelEMA(student_model, ema)

no_decay = ['bn']
weight_decay = 5e-4 # default 0 / mainargs 5e-4
teacher_parameters = [{'params': [p for n, p in teacher_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in teacher_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
student_parameters = [{'params': [p for n, p in student_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in student_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

# teacher_model.zero_grad()
# student_model.zero_grad()





In [ ]:
# @title try
# import torch
# import torch.nn as nn
# from torchvision import models
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # # 1.9, 1.7, 1.2
# im = torch.rand(32,3,400,680,device=device) #
# # print(32*3*400*680*32/8)
# # print(im.element_size() * im.nelement())

# def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)
# # print(count_parameters(model))


# model = models.resnet152(weights='DEFAULT') # 18 34 50 101 152
# model.fc = nn.Sequential()
# model = model.to(device)
# # model = torch.compile(model.to(device))
# # 32, 50:3.3-5=1.7
# # print(count_parameters(model))

# # amp res152 student cant train on batch 32, compile or not

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# model.eval()
# student_model.eval()
# with torch.cuda.amp.autocast():
#     with torch.no_grad(): #
#         # out = model(im) # 1:2.7, 4:6.3, 8:12.8
#         out = student_model(im) #
# # 16*15: 11.6
# 32:1.3-8.7=6.4

# student_model.train()
# out = student_model(im) #

# model.train()
# with torch.cuda.amp.autocast():
#     out = model(im) #


# # 152:3.2
# # im = torch.rand(4,3,400,680,device=device) #
# # out = model(im) # 5.9

# im = torch.rand(16,2048,device=device) #
# out = teacher_model(im) # 5.9

# torch.cuda.empty_cache()


In [7]:
# @title wandb
# https://docs.wandb.ai/quickstart
!pip install wandb
import wandb
wandb.login() # 487a2109e55dce4e13fc70681781de9f50f27be7
run = wandb.init(
    project="mpl",
    config={
        "optim": "adamw",
        # "lr": lr,
        # "epochs": epochs,
    })


wandb: Currently logged in as: bobdole. Use `wandb login --relogin` to force relogin


In [ ]:
# @title main mpl train
# ########## https://github.com/kekmodel/MPL-pytorch/blob/main/main.py
# import random
# import numpy as np
# import torch
# from torch.cuda import amp
# from torch import nn
# from torch.nn import functional as F

# ema = 0.995 # default 0 / mainargs 0.995
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True

# t_scaler = amp.GradScaler()
# s_scaler = amp.GradScaler()
# def train(labeled_loader, unlabeled_loader, teacher_model, student_model,
#         avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler):
#     labeled_iter = iter(labeled_loader)
#     unlabeled_iter = iter(unlabeled_loader)

#     # for author's code formula
#     # moving_dot_product = torch.empty(1).to(device)
#     # limit = 3.0**(0.5)  # 3 = 6 / (f_in + f_out)
#     # nn.init.uniform_(moving_dot_product, -limit, limit)

#     # eval_step = 10#1000
#     # start_step=0
#     # for step in range(start_step, total_steps):
#     for step in range(len(unlabeled_loader)):
#         teacher_model.train()
#         student_model.train()

#         try: images_l, targets = next(labeled_iter)
#         except:
#             labeled_iter = iter(labeled_loader)
#             images_l, targets = next(labeled_iter)
#         try: images_uw = next(unlabeled_iter) # images_uw, _ = next(unlabeled_iter)
#         except:
#             unlabeled_iter = iter(unlabeled_loader)
#             images_uw = next(unlabeled_iter) # me
#         images_l, targets = images_l.to(device), targets.to(device)
#         # images_uw, images_us = images_uw.to(device), images_us.to(device)
#         images_uw = images_uw.to(device)
#         images_us = trs(images_uw)

#         with amp.autocast():
#             batch_size = images_l.shape[0]
#             # print(images_l.shape, images_uw.shape, images_us.shape) # [16, 3, 400, 640], [112, 3, 400, 640], [112, 3, 400, 640] mu*batch_size
#             t_images = torch.cat((images_l, images_uw, images_us))
#             t_logits = teacher_model(t_images)
#             t_logits_l = t_logits[:batch_size]
#             t_logits_uw, t_logits_us = t_logits[batch_size:].chunk(2)
#             del t_logits

#             t_loss_l = criterion(t_logits_l, targets)

#             temperature = 1 # default 1 / mainargs 0.7
#             soft_pseudo_label = torch.softmax(t_logits_uw.detach() / temperature, dim=-1)
#             max_probs, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1)

#             threshold = 0.95 # default 0.95 / mainargs 0.6
#             mask = max_probs.ge(threshold).float()
#             t_loss_u = torch.mean(-(soft_pseudo_label * torch.log_softmax(t_logits_us, dim=-1)).sum(dim=-1) * mask)
#             lambda_u = 8 # default 1 / mainargs 8 coefficient of unlabeled loss
#             uda_steps = 10 # default 1 / mainargs 5000 warmup steps of lambda-u
#             weight_u = lambda_u * min(1., (step + 1) / uda_steps)
#             t_loss_uda = t_loss_l + weight_u * t_loss_u

#             s_images = torch.cat((images_l, images_us))
#             s_logits = student_model(s_images)
#             s_logits_l = s_logits[:batch_size]
#             s_logits_us = s_logits[batch_size:]
#             del s_logits

#             s_loss_l_old = F.cross_entropy(s_logits_l.detach(), targets)

#             # print("s_logits_us, hard_pseudo_label: ", s_logits_us.shape, hard_pseudo_label.shape) # [448, 10] [224]
#             s_loss = criterion(s_logits_us, hard_pseudo_label)

#         s_scaler.scale(s_loss).backward()

#         # if grad_clip > 0:
#         s_scaler.unscale_(s_optimizer)
#         nn.utils.clip_grad_norm_(student_model.parameters(), 1e9)

#         s_scaler.step(s_optimizer)
#         s_scaler.update()
#         s_scheduler.step()

#         if ema > 0: avg_student_model.update_parameters(student_model)

#         with amp.autocast():
#             with torch.no_grad():
#                 s_logits_l = student_model(images_l)
#             s_loss_l_new = F.cross_entropy(s_logits_l.detach(), targets)

#             dot_product = s_loss_l_old - s_loss_l_new # theoretically correct formula (https://github.com/kekmodel/MPL-pytorch/issues/6)
#             # dot_product = s_loss_l_new - s_loss_l_old # author's code formula
#             # # moving_dot_product = moving_dot_product * 0.99 + dot_product * 0.01
#             # # dot_product = dot_product - moving_dot_product

#             _, hard_pseudo_label = torch.max(t_logits_us.detach(), dim=-1)
#             t_loss_mpl = dot_product * F.cross_entropy(t_logits_us, hard_pseudo_label)

#             # t_loss_mpl = torch.tensor(0.).to(device) # test
#             t_loss = t_loss_uda + t_loss_mpl

#         t_scaler.scale(t_loss).backward()

#         # if grad_clip > 0:
#         t_scaler.unscale_(t_optimizer)
#         nn.utils.clip_grad_norm_(teacher_model.parameters(), 1e9)

#         t_scaler.step(t_optimizer)
#         t_scaler.update()
#         t_scheduler.step()

#         teacher_model.zero_grad()
#         student_model.zero_grad()

#         # if (step + 1) % eval_step == 0:
#         #     # print(s_losses, t_losses, t_losses_l, t_losses_u, t_losses_mpl, mean_mask)
#         #     test_model = avg_student_model if avg_student_model is not None else student_model
#         #     test_loss, top1, top5 = evaluate(test_loader, test_model, criterion)
#     return



In [ ]:
# @title mpl try 1
# # https://github.com/kekmodel/MPL-pytorch/blob/main/main.py
# import random
# import numpy as np
# import torch
# from torch.cuda import amp
# from torch import nn
# from torch.nn import functional as F

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True

# t_scaler = amp.GradScaler()
# s_scaler = amp.GradScaler()
# def train(labeled_loader, unlabeled_loader, teacher_model, student_model,
#         avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler):
#     labeled_iter = iter(labeled_loader)
#     unlabeled_iter = iter(unlabeled_loader)
#     size = len(unlabeled_loader)

#     # for author's code formula
#     # moving_dot_product = torch.empty(1).to(device)
#     # limit = 3.0**(0.5)  # 3 = 6 / (f_in + f_out)
#     # nn.init.uniform_(moving_dot_product, -limit, limit)

#     teacher_model.train()
#     student_model.train()
#     # eval_step = 10#1000
#     # start_step=0
#     # for step in range(start_step, total_steps):
#     for step in range(len(unlabeled_loader)):

#         try: images_l, targets = next(labeled_iter)
#         except:
#             labeled_iter = iter(labeled_loader)
#             images_l, targets = next(labeled_iter)
#         try: images_uw = next(unlabeled_iter) # images_uw, _ = next(unlabeled_iter)
#         except:
#             unlabeled_iter = iter(unlabeled_loader)
#             images_uw = next(unlabeled_iter) # me
#         images_l, targets = images_l.to(device), targets.to(device)
#         images_uw = images_uw.to(device)
#         images_us = trs(images_uw)

#         with amp.autocast():
#             batch_size = images_l.shape[0]

#             t_images = torch.cat((images_l, images_us))
#             with torch.no_grad(): t_images = model(t_images) # reduced pml # big teacher
#             t_logits = teacher_model(t_images)
#             t_logits_l, t_logits_us = t_logits[:batch_size], t_logits[batch_size:]
#             del t_logits
#             torch.cuda.empty_cache()
#             with torch.no_grad(): # t_logits_uw no need grad
#                 images_uw = model(images_uw) # reduced pml # big teacher
#                 t_logits_uw = teacher_model(images_uw)
#             torch.cuda.empty_cache()
#             temperature = 1 # default 1 / mainargs 0.7
#             # soft_pseudo_label = torch.softmax(t_logits_uw.detach() / temperature, dim=-1)
#             soft_pseudo_label = torch.softmax(t_logits_uw / temperature, dim=-1)
#             max_probs, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1)


#         with amp.autocast():
#             t_loss_l = criterion(t_logits_l, targets)
#         # t_scaler.scale(t_loss_l).backward() # me backward first

#         with amp.autocast():
#             threshold = 0.95 # default 0.95 / mainargs 0.6
#             mask = max_probs.ge(threshold).float()
#             t_loss_u = torch.mean(-(soft_pseudo_label * torch.log_softmax(t_logits_us, dim=-1)).sum(dim=-1) * mask)
#             lambda_u = 8 # default 1 / mainargs 8 coefficient of unlabeled loss
#             uda_steps = 10 # default 1 / mainargs 5000 warmup steps of lambda-u
#             weight_u = lambda_u * min(1., (step + 1) / uda_steps)
#             t_loss_uda = t_loss_l + weight_u * t_loss_u
#         # t_scaler.scale(weight_u * t_loss_u).backward()
#         # del soft_pseudo_label, t_loss_u



#         with amp.autocast():
#             s_logits_us = student_model(images_us)
#             with torch.no_grad():
#                 s_logits_l = student_model(images_l)
#             # s_loss_l_old = F.cross_entropy(s_logits_l.detach(), targets)
#             s_loss_l_old = F.cross_entropy(s_logits_l, targets)
#             s_loss = criterion(s_logits_us, hard_pseudo_label)
#         s_scaler.scale(s_loss).backward()


#         # if grad_clip > 0:
#         s_scaler.unscale_(s_optimizer)
#         nn.utils.clip_grad_norm_(student_model.parameters(), 1e9)
#         s_scaler.step(s_optimizer)
#         s_scaler.update()
#         s_scheduler.step()
#         student_model.zero_grad()
#         if ema > 0: avg_student_model.update_parameters(student_model)





#         with amp.autocast():
#             with torch.no_grad():
#                 s_logits_l = student_model(images_l)
#             # s_loss_l_new = F.cross_entropy(s_logits_l.detach(), targets)
#             s_loss_l_new = F.cross_entropy(s_logits_l, targets)

#             dot_product = s_loss_l_old - s_loss_l_new # theoretically correct formula (https://github.com/kekmodel/MPL-pytorch/issues/6)
#             # dot_product = s_loss_l_new - s_loss_l_old # author's code formula
#             # # moving_dot_product = moving_dot_product * 0.99 + dot_product * 0.01
#             # # dot_product = dot_product - moving_dot_product

#             _, hard_pseudo_label = torch.max(t_logits_us.detach(), dim=-1)
#             t_loss_mpl = dot_product * F.cross_entropy(t_logits_us, hard_pseudo_label) # dot_product no grad

#             # t_loss_mpl = torch.tensor(0.).to(device) # test
#             t_loss = t_loss_uda + t_loss_mpl
#         t_scaler.scale(t_loss).backward()
#         print("t_loss, s_loss", t_loss.item(), s_loss.item())
#         # t_scaler.scale(t_loss_mpl).backward()


#         t_scaler.unscale_(t_optimizer) # if grad_clip > 0:
#         nn.utils.clip_grad_norm_(teacher_model.parameters(), 1e9)
#         t_scaler.step(t_optimizer)
#         t_scaler.update()
#         t_scheduler.step()
#         teacher_model.zero_grad()

#     return



In [ ]:
# @title mpl try
# # https://github.com/kekmodel/MPL-pytorch/blob/main/main.py
# import random
# import numpy as np
# import torch
# from torch.cuda import amp
# from torch import nn
# from torch.nn import functional as F

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True

# t_scaler = amp.GradScaler()
# s_scaler = amp.GradScaler()
# def train(labeled_loader, unlabeled_loader, teacher_model, student_model,
#         avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler):
#     labeled_iter = iter(labeled_loader)
#     unlabeled_iter = iter(unlabeled_loader)
#     size = len(unlabeled_loader)

#     # for author's code formula
#     # moving_dot_product = torch.empty(1).to(device)
#     # limit = 3.0**(0.5)  # 3 = 6 / (f_in + f_out)
#     # nn.init.uniform_(moving_dot_product, -limit, limit)

#     teacher_model.train()
#     student_model.train()
#     # eval_step = 10#1000
#     # start_step=0
#     # for step in range(start_step, total_steps):
#     for step in range(len(unlabeled_loader)):

#         try: images_l, targets = next(labeled_iter)
#         except:
#             labeled_iter = iter(labeled_loader)
#             images_l, targets = next(labeled_iter)
#         try: images_uw = next(unlabeled_iter) # images_uw, _ = next(unlabeled_iter)
#         except:
#             unlabeled_iter = iter(unlabeled_loader)
#             images_uw = next(unlabeled_iter) # me
#         images_l, targets = images_l.to(device), targets.to(device)
#         images_uw = images_uw.to(device)
#         images_us = trs(images_uw)

#         with amp.autocast():
#             batch_size = images_l.shape[0]

#             t_images = torch.cat((images_l, images_us))
#             with torch.no_grad(): t_images = model(t_images) # reduced pml # big teacher
#             t_logits = teacher_model(t_images)
#             t_logits_l, t_logits_us = t_logits[:batch_size], t_logits[batch_size:]
#             del t_logits, t_images
#             torch.cuda.empty_cache()
#             with torch.no_grad(): # t_logits_uw no need grad
#                 images_uw = model(images_uw) # reduced pml # big teacher
#                 t_logits_uw = teacher_model(images_uw)
#             torch.cuda.empty_cache()
#             temperature = 1 # default 1 / mainargs 0.7
#             # soft_pseudo_label = torch.softmax(t_logits_uw.detach() / temperature, dim=-1)
#             soft_pseudo_label = torch.softmax(t_logits_uw / temperature, dim=-1)
#             max_probs, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1) # all no grads


#         with amp.autocast():
#             t_loss_l = criterion(t_logits_l, targets)
#         t_scaler.scale(t_loss_l).backward(retain_graph=True) # me backward first
#         del t_loss_l

#         with amp.autocast():
#             threshold = 0.95 # default 0.95 / mainargs 0.6
#             mask = max_probs.ge(threshold).float()
#             t_loss_u = torch.mean(-(soft_pseudo_label * torch.log_softmax(t_logits_us, dim=-1)).sum(dim=-1) * mask)
#             lambda_u = 8 # default 1 / mainargs 8 coefficient of unlabeled loss
#             uda_steps = 10 # default 1 / mainargs 5000 warmup steps of lambda-u
#             weight_u = lambda_u * min(1., (step + 1) / uda_steps)
#             # t_loss_uda = t_loss_l + weight_u * t_loss_u
#         t_scaler.scale(weight_u * t_loss_u).backward(retain_graph=True)
#         del t_loss_u, soft_pseudo_label# t_loss_l
#         torch.cuda.empty_cache()



#         with amp.autocast():
#             s_logits_us = student_model(images_us)
#             with torch.no_grad():
#                 s_logits_l = student_model(images_l)
#             # s_loss_l_old = F.cross_entropy(s_logits_l.detach(), targets)
#             s_loss_l_old = F.cross_entropy(s_logits_l, targets)
#             s_loss = criterion(s_logits_us, hard_pseudo_label)
#         s_scaler.scale(s_loss).backward()
#         del s_logits_l, s_logits_us, hard_pseudo_label#, s_loss
#         torch.cuda.empty_cache()


#         # if grad_clip > 0:
#         s_scaler.unscale_(s_optimizer)
#         nn.utils.clip_grad_norm_(student_model.parameters(), 1e9)
#         s_scaler.step(s_optimizer)
#         s_scaler.update()
#         s_scheduler.step()
#         student_model.zero_grad()
#         if ema > 0: avg_student_model.update_parameters(student_model)





#         with amp.autocast():
#             with torch.no_grad():
#                 s_logits_l = student_model(images_l)
#             # s_loss_l_new = F.cross_entropy(s_logits_l.detach(), targets)
#             s_loss_l_new = F.cross_entropy(s_logits_l, targets)

#             dot_product = s_loss_l_old - s_loss_l_new # theoretically correct formula (https://github.com/kekmodel/MPL-pytorch/issues/6)
#             # dot_product = s_loss_l_new - s_loss_l_old # author's code formula
#             # # moving_dot_product = moving_dot_product * 0.99 + dot_product * 0.01
#             # # dot_product = dot_product - moving_dot_product

#             _, hard_pseudo_label = torch.max(t_logits_us.detach(), dim=-1)
#             t_loss_mpl = dot_product * F.cross_entropy(t_logits_us, hard_pseudo_label) # dot_product no grad

#             # t_loss_mpl = torch.tensor(0.).to(device) # test
#             # t_loss = t_loss_uda + t_loss_mpl
#         # t_scaler.scale(t_loss).backward()
#         t_scaler.scale(t_loss_mpl).backward()
#         # print("t_loss, s_loss", t_loss.item(), s_loss.item())
#         # t_scaler.scale(t_loss_mpl).backward()
#         del s_logits_l, dot_product, s_loss_l_old, s_loss_l_new, t_logits_us, hard_pseudo_label, #t_loss, t_loss_uda, t_loss_mpl
#         del s_loss
#         torch.cuda.empty_cache()


#         t_scaler.unscale_(t_optimizer) # if grad_clip > 0:
#         nn.utils.clip_grad_norm_(teacher_model.parameters(), 1e9)
#         t_scaler.step(t_optimizer)
#         t_scaler.update()
#         t_scheduler.step()
#         teacher_model.zero_grad()

#     return



In [ ]:
# @title mpl looop mu
# # https://github.com/kekmodel/MPL-pytorch/blob/main/main.py
# import random
# import numpy as np
# import torch
# from torch.cuda import amp
# from torch import nn
# from torch.nn import functional as F

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True

# t_scaler = amp.GradScaler()
# s_scaler = amp.GradScaler()
# def train(labeled_loader, unlabeled_loader, teacher_model, student_model,
#         avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler):
#     labeled_iter = iter(labeled_loader)
#     unlabeled_iter = iter(unlabeled_loader)
#     size = len(unlabeled_loader)

#     # for author's code formula
#     # moving_dot_product = torch.empty(1).to(device)
#     # limit = 3.0**(0.5)  # 3 = 6 / (f_in + f_out)
#     # nn.init.uniform_(moving_dot_product, -limit, limit)

#     teacher_model.train()
#     student_model.train()
#     for step in range(len(unlabeled_loader)):

#         try: images_l, targets = next(labeled_iter)
#         except:
#             labeled_iter = iter(labeled_loader)
#             images_l, targets = next(labeled_iter)
#         try: images_uw = next(unlabeled_iter) # images_uw, _ = next(unlabeled_iter)
#         except:
#             unlabeled_iter = iter(unlabeled_loader)
#             images_uw = next(unlabeled_iter) # me
#         images_l, targets = images_l.to(device), targets.to(device)
#         images_uw = images_uw.to(device)
#         images_us = trs(images_uw)

#         # torch.cuda.empty_cache()
#         with amp.autocast():
#             batch_size = images_l.shape[0]

#             with torch.no_grad(): m_logits_l = model(images_l) # reduced pml # big teacher
#             t_logits_l = teacher_model(m_logits_l)
#             t_loss_l = criterion(t_logits_l, targets)
#         # t_scaler.scale(t_loss_l).backward(retain_graph=True) # me backward first
#         t_scaler.scale(t_loss_l).backward() # me backward first
#         # del t_loss_l



#         # once, grad:s_loss_l_old
#         with amp.autocast():
#             with torch.no_grad(): s_logits_l = student_model(images_l)
#             s_loss_l_old = F.cross_entropy(s_logits_l, targets)

#         s_loss = 0
#         t_loss_uda = t_loss_l
#         t_logits_us = torch.empty(0, device=device)
#         for i_us, i_uw in zip(images_us.chunk(mu), images_uw.chunk(mu)):
#             with amp.autocast():
#                 with torch.no_grad():
#                     m_i_us = model(i_us) # reduced pml # big teacher
#                     m_i_uw = model(i_uw) # reduced pml # big teacher
#                     t_l_uw = teacher_model(m_i_uw) # t_logits_uw no need grad
#                 t_l_us = teacher_model(m_i_us)
#                 t_logits_us = torch.cat((t_logits_us, t_l_us))

#                 temperature = 1 # default 1 / mainargs 0.7
#                 # soft_pseudo_label = torch.softmax(t_logits_uw.detach() / temperature, dim=-1)
#                 soft_pseudo_label = torch.softmax(t_l_uw / temperature, dim=-1)
#                 max_probs, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1) # all no grads

#                 threshold = 0.95 # default 0.95 / mainargs 0.6
#                 mask = max_probs.ge(threshold).float()
#                 t_loss_u = torch.mean(-(soft_pseudo_label * torch.log_softmax(t_l_us, dim=-1)).sum(dim=-1) * mask)
#                 lambda_u = 8 # default 1 / mainargs 8 coefficient of unlabeled loss
#                 uda_steps = 10 # default 1 / mainargs 5000 warmup steps of lambda-u
#                 weight_u = lambda_u * min(1., (step + 1) / uda_steps)
#                 # t_loss_uda = t_loss_l + weight_u * t_loss_u

#                 # i_us.retain_grad()
#                 i_us.requires_grad=True
#                 s_l_us = student_model(i_us)
#                 s_l = criterion(s_l_us, hard_pseudo_label)
#             s_scaler.scale(s_l).backward()
#             s_loss += s_l
#             t_scaler.scale(weight_u * t_loss_u).backward(retain_graph=True)
#             # t_scaler.scale(weight_u * t_loss_u).backward()
#             t_loss_uda += weight_u * t_loss_u
#         # del t_loss_u, soft_pseudo_label# t_loss_l
#         # del s_logits_l, s_logits_us, hard_pseudo_label#, s_loss



#         # if grad_clip > 0:
#         s_scaler.unscale_(s_optimizer)
#         nn.utils.clip_grad_norm_(student_model.parameters(), 1e9)
#         s_scaler.step(s_optimizer)
#         s_scaler.update()
#         s_scheduler.step()
#         student_model.zero_grad()
#         if ema > 0: avg_student_model.update_parameters(student_model)

#         # once, grad:t_logits_us, s_loss_l_old
#         with amp.autocast():
#             with torch.no_grad(): s_logits_l = student_model(images_l)
#             s_loss_l_new = F.cross_entropy(s_logits_l, targets)
#             dot_product = s_loss_l_old - s_loss_l_new
#             # # moving_dot_product = moving_dot_product * 0.99 + dot_product * 0.01
#             # # dot_product = dot_product - moving_dot_product
#         t_loss_mpl = 0
#         for t_l_us in t_logits_us.chunk(mu):
#             with amp.autocast():
#                 _, hard_pseudo_label = torch.max(t_l_us.detach(), dim=-1)
#                 t_l_mpl = dot_product * F.cross_entropy(t_l_us, hard_pseudo_label) # dot_product no grad
#             t_scaler.scale(t_l_mpl).backward(retain_graph=True)
#             t_loss_mpl += t_l_mpl

#         t_loss = t_loss_uda + t_loss_mpl
#         # print("t_loss, s_loss", t_loss.item(), s_loss.item())
#         if step%10==0: print(step,"/",size," ", "t_loss: ", t_loss.item(), "s_loss: ", s_loss.item())
#         try: wandb.log({"t_loss": t_loss.item(), "s_loss": s_loss.item()})
#         except: pass
#         # del s_logits_l, dot_product, s_loss_l_old, s_loss_l_new, t_logits_us, hard_pseudo_label, #t_loss, t_loss_uda, t_loss_mpl
#         # del s_loss


#         t_scaler.unscale_(t_optimizer) # if grad_clip > 0:
#         nn.utils.clip_grad_norm_(teacher_model.parameters(), 1e9)
#         t_scaler.step(t_optimizer)
#         t_scaler.update()
#         t_scheduler.step()
#         teacher_model.zero_grad()

#     return



In [8]:
# @title mpl grad acc
# https://github.com/kekmodel/MPL-pytorch/blob/main/main.py
import random
import numpy as np
import torch
from torch.cuda import amp
from torch import nn
from torch.nn import functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

t_scaler = amp.GradScaler()
s_scaler = amp.GradScaler()
def train(labeled_loader, unlabeled_loader, teacher_model, student_model,
        avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler):
    labeled_iter = iter(labeled_loader)
    unlabeled_iter = iter(unlabeled_loader)
    size = len(unlabeled_loader)

    # for author's code formula
    # moving_dot_product = torch.empty(1).to(device)
    # limit = 3.0**(0.5)  # 3 = 6 / (f_in + f_out)
    # nn.init.uniform_(moving_dot_product, -limit, limit)

    teacher_model.train()
    student_model.train()
    for step in range(len(unlabeled_loader)):

        try: cimages_l, ctargets = next(labeled_iter)
        except:
            labeled_iter = iter(labeled_loader)
            cimages_l, ctargets = next(labeled_iter)
        try: cimages_uw = next(unlabeled_iter) # images_uw, _ = next(unlabeled_iter)
        except:
            unlabeled_iter = iter(unlabeled_loader)
            cimages_uw = next(unlabeled_iter) # me
        cimages_l, ctargets = cimages_l.to(device), ctargets.to(device)
        cimages_uw = cimages_uw.to(device)
        cimages_us = trs(cimages_uw)

        for images_l, targets, images_uw, images_us in zip(cimages_l.chunk(grad_acc), ctargets.chunk(grad_acc), cimages_uw.chunk(grad_acc), cimages_us.chunk(grad_acc)): # for grad acc 1/2

            with amp.autocast():
                batch_size = images_l.shape[0]

                with torch.no_grad(): m_logits_l = model(images_l) # reduced pml # big teacher
                t_logits_l = teacher_model(m_logits_l)
                t_loss_l = criterion(t_logits_l, targets)
            # t_scaler.scale(t_loss_l).backward(retain_graph=True) # me backward first
            t_scaler.scale(t_loss_l).backward() # me backward first
            # del t_loss_l


            with amp.autocast():
                with torch.no_grad(): s_logits_l = student_model(images_l)
                s_loss_l_old = F.cross_entropy(s_logits_l, targets)

            s_loss = 0
            t_loss_uda = t_loss_l
            t_logits_us = torch.empty(0, device=device)
            for i_us, i_uw in zip(images_us.chunk(mu), images_uw.chunk(mu)):
                with amp.autocast():
                    with torch.no_grad():
                        m_i_us = model(i_us) # reduced pml # big teacher
                        m_i_uw = model(i_uw) # reduced pml # big teacher
                        t_l_uw = teacher_model(m_i_uw) # t_logits_uw no need grad
                    t_l_us = teacher_model(m_i_us)
                    t_logits_us = torch.cat((t_logits_us, t_l_us))

                    temperature = 1 # default 1 / mainargs 0.7
                    # soft_pseudo_label = torch.softmax(t_logits_uw.detach() / temperature, dim=-1)
                    soft_pseudo_label = torch.softmax(t_l_uw / temperature, dim=-1)
                    max_probs, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1) # all no grads

                    threshold = 0.95 # default 0.95 / mainargs 0.6
                    mask = max_probs.ge(threshold).float()
                    t_loss_u = torch.mean(-(soft_pseudo_label * torch.log_softmax(t_l_us, dim=-1)).sum(dim=-1) * mask)
                    lambda_u = 8 # default 1 / mainargs 8 coefficient of unlabeled loss
                    uda_steps = 10 # default 1 / mainargs 5000 warmup steps of lambda-u
                    weight_u = lambda_u * min(1., (step + 1) / uda_steps)
                    # t_loss_uda = t_loss_l + weight_u * t_loss_u

                    # i_us.retain_grad()
                    i_us.requires_grad=True
                    s_l_us = student_model(i_us)
                    s_l = criterion(s_l_us, hard_pseudo_label)
                s_scaler.scale(s_l).backward()
                s_loss += s_l
                t_scaler.scale(weight_u * t_loss_u).backward(retain_graph=True)
                # t_scaler.scale(weight_u * t_loss_u).backward()
                t_loss_uda += weight_u * t_loss_u
            # del t_loss_u, soft_pseudo_label# t_loss_l
            # del s_logits_l, s_logits_us, hard_pseudo_label#, s_loss



        # if grad_clip > 0:
        s_scaler.unscale_(s_optimizer)
        nn.utils.clip_grad_norm_(student_model.parameters(), 1e9)
        s_scaler.step(s_optimizer)
        s_scaler.update()
        s_scheduler.step()
        student_model.zero_grad()
        if ema > 0: avg_student_model.update_parameters(student_model)


        for images_l, targets, images_uw, images_us in zip(cimages_l.chunk(grad_acc), ctargets.chunk(grad_acc), cimages_uw.chunk(grad_acc), cimages_us.chunk(grad_acc)): # for grad acc 2/2

            with amp.autocast():
                with torch.no_grad(): s_logits_l = student_model(images_l)
                s_loss_l_new = F.cross_entropy(s_logits_l, targets)
                dot_product = s_loss_l_old - s_loss_l_new
                # # moving_dot_product = moving_dot_product * 0.99 + dot_product * 0.01
                # # dot_product = dot_product - moving_dot_product
            t_loss_mpl = 0
            for t_l_us in t_logits_us.chunk(mu):
                with amp.autocast():
                    _, hard_pseudo_label = torch.max(t_l_us.detach(), dim=-1)
                    t_l_mpl = dot_product * F.cross_entropy(t_l_us, hard_pseudo_label) # dot_product no grad
                t_scaler.scale(t_l_mpl).backward(retain_graph=True)
                t_loss_mpl += t_l_mpl

            t_loss = t_loss_uda + t_loss_mpl
            # print("t_loss, s_loss", t_loss.item(), s_loss.item())
            if step%10==0: print(step,"/",size," ", "t_loss: ", t_loss.item(), "s_loss: ", s_loss.item())
            try: wandb.log({"t_loss": t_loss.item(), "s_loss": s_loss.item()})
            except: pass
            # del s_logits_l, dot_product, s_loss_l_old, s_loss_l_new, t_logits_us, hard_pseudo_label, #t_loss, t_loss_uda, t_loss_mpl
            # del s_loss




        t_scaler.unscale_(t_optimizer) # if grad_clip > 0:
        nn.utils.clip_grad_norm_(teacher_model.parameters(), 1e9)
        t_scaler.step(t_optimizer)
        t_scaler.update()
        t_scheduler.step()
        teacher_model.zero_grad()

    return



In [9]:
# @title strain eval

def evaluate(test_loader, model, criterion, verbose=True):
    size = len(test_loader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for step, (images, targets) in enumerate(test_loader):
            batch_size = images.shape[0]
            images = images.to(device)
            targets = targets.to(device)
            with amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)
            test_loss += loss.item()
        #     acc1, acc5 = accuracy(outputs, targets, (1, 5))
        #     losses, top1, top5 = loss.item(), acc1[0], acc5[0]
        # # return losses, top1, top5
            correct += (outputs.argmax(1) == targets).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    try: wandb.log({"test loss": test_loss})
    except: pass
    if verbose: print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss


# @title train test function
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()
# https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/resnet_new.py
from torch.utils.checkpoint import checkpoint, checkpoint_sequential

trs=TrainTransform() # for image augmentation during train time
# train function with automatic mixed precision
def strain(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    size = len(dataloader.dataset)
    model.train()
    loss_list = []
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        with torch.cuda.amp.autocast(): # automatic mixed percision
            x = trs(x) # image augmentation during train time to use gpu
            pred = model(x) # default
            loss = loss_fn(pred, y)
        scaler.scale(loss).backward()
        if ((batch + 1) % 4 == 0) or (batch + 1 == len(dataloader)): # gradient accumulation
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
                # print("### lr: ", optimizer.param_groups[0]["lr"])

        # print(model.state_dict()['_orig_mod.bn1.running_mean'][0])
        train_loss = loss.item()/len(y)
        loss_list.append(train_loss)
        try: wandb.log({"train loss": train_loss})
        except: pass
        # if batch % (size//(10* len(y))) == 0:
        current = batch * len(x)
        if verbose: print(f"loss: {train_loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss_list



In [10]:
# @title wwwwwwwwwww

criterion = create_loss_fn()

from torch import optim
# lr default 0.01/ mainargs 0.05
# og:t0.05s0.05 , psl:t1e-3s3e-4, rpsl:t
# t_optimizer = optim.SGD(teacher_parameters, lr=0.05, momentum=0.9, nesterov=True)
# s_optimizer = optim.SGD(student_parameters, lr=0.05, momentum=0.9, nesterov=True)
t_optimizer = optim.SGD(teacher_parameters, lr=1e-6, momentum=0.9, nesterov=True)
s_optimizer = optim.SGD(student_parameters, lr=1e-5, momentum=0.9, nesterov=True)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.9, 0.999), optim_bits=8)
# optimizer = Lamb(model.parameters(), lr=1e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)

# 3e-5 27.9->25.0
# 3e-3 27.9->26.5
# 0.05 29.4 -> 22.1

epochs = 5
num_batches=len(train_loader)
total_steps=int(np.ceil(num_batches/grad_acc)*epochs +1) # +1 to excluse uptick at the end of onecycle
# total_steps=30 # 300000
warmup_steps = 10 # default 0 / mainargs 5000
# t_scheduler = get_cosine_schedule_with_warmup(t_optimizer, warmup_steps, total_steps)
t_scheduler = get_cosine_schedule_with_warmup(t_optimizer, warmup_steps, total_steps,10)
student_wait_steps = 6 # default 0 / mainargs 3000
s_scheduler = get_cosine_schedule_with_warmup(s_optimizer, warmup_steps, total_steps, student_wait_steps)



In [ ]:

# scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=int(np.ceil(num_batches/4)*3), power=1.0)
# scheduler = PolynomialLR(optimizer, total_iters=4, power=1.0)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=10**(-1/2))


# t_optimizer.param_groups[0]["lr"]
# s_optimizer.param_groups[0]["lr"]

# import time
# start = time.time()

# pth='/content/mpl.pth' # ty
pth='/content/drive/MyDrive/frame/mpl18.pth' # M

for t in range(0,epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    t_lr=t_optimizer.param_groups[0]["lr"]
    s_lr=s_optimizer.param_groups[0]["lr"]
    print('t_lr,s_lr',t_lr,s_lr)
    train(labeled_loader, unlabeled_loader, teacher_model, student_model,
        avg_student_model, criterion, t_optimizer, s_optimizer, t_scheduler, s_scheduler)

    # evaluate(test_loader, student_model, criterion)
    evaluate(test_loader, avg_student_model, criterion)

    checkpoint = {
    'epoch': t+1,
    'teacher_model': teacher_model.state_dict(),
    'student_model': student_model.state_dict(),
    'avg_student_model': avg_student_model.state_dict(),
    't_optimizer': t_optimizer.state_dict(),
    's_optimizer': s_optimizer.state_dict(),
    't_scheduler': t_scheduler.state_dict(),
    's_scheduler': s_scheduler.state_dict(),}
    torch.save(checkpoint, pth)


# res34, batch4 8.8
# res18, batch16 11.6 nocompilemodel

# 16m28s


Epoch 1
-------------------------------
t_lr,s_lr 0.0 0.0


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


0 / 157   t_loss:  1.7915688753128052 s_loss:  12.792863845825195


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


10 / 157   t_loss:  1.790928840637207 s_loss:  12.847481727600098
20 / 157   t_loss:  1.7904629707336426 s_loss:  12.796439170837402
30 / 157   t_loss:  1.7913095951080322 s_loss:  12.93379020690918
40 / 157   t_loss:  1.7897231578826904 s_loss:  12.644171714782715
50 / 157   t_loss:  1.7884248495101929 s_loss:  12.857980728149414
60 / 157   t_loss:  1.7886351346969604 s_loss:  12.7106351852417
70 / 157   t_loss:  1.789746642112732 s_loss:  12.616300582885742
80 / 157   t_loss:  1.788618803024292 s_loss:  12.452911376953125
90 / 157   t_loss:  1.7908527851104736 s_loss:  12.49996566772461
100 / 157   t_loss:  1.789739966392517 s_loss:  12.630684852600098
110 / 157   t_loss:  1.787540078163147 s_loss:  12.507417678833008
120 / 157   t_loss:  1.7870131731033325 s_loss:  12.475946426391602
130 / 157   t_loss:  1.7873234748840332 s_loss:  12.391185760498047
140 / 157   t_loss:  1.788748025894165 s_loss:  12.291733741760254
150 / 157   t_loss:  1.787776231765747 s_loss:  12.426130294799805


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


0 / 157   t_loss:  1.7903753519058228 s_loss:  12.253273963928223
10 / 157   t_loss:  1.7899593114852905 s_loss:  12.10791015625
20 / 157   t_loss:  1.7883033752441406 s_loss:  12.418424606323242
30 / 157   t_loss:  1.7867603302001953 s_loss:  12.069341659545898
40 / 157   t_loss:  1.7882957458496094 s_loss:  11.937756538391113
50 / 157   t_loss:  1.7863802909851074 s_loss:  12.063082695007324
60 / 157   t_loss:  1.7878957986831665 s_loss:  12.109430313110352
70 / 157   t_loss:  1.7880655527114868 s_loss:  12.139351844787598
80 / 157   t_loss:  1.7887767553329468 s_loss:  11.826499938964844
90 / 157   t_loss:  1.7870361804962158 s_loss:  11.899945259094238
100 / 157   t_loss:  1.7845218181610107 s_loss:  11.708991050720215
110 / 157   t_loss:  1.789860486984253 s_loss:  11.788382530212402
120 / 157   t_loss:  1.7882481813430786 s_loss:  11.626124382019043
130 / 157   t_loss:  1.7890647649765015 s_loss:  11.756231307983398
140 / 157   t_loss:  1.7867783308029175 s_loss:  11.451495170593

In [ ]:


# finetune
# model = student_model
model = avg_student_model
model.drop = nn.Identity()
# labeled_loader = DataLoader(finetune_dataset, batch_size=128, num_workers=4, pin_memory=True) # batch_size=512
labeled_loader = DataLoader(finetune_dataset, batch_size=128, pin_memory=True) # batch_size=512
optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum=0.9, weight_decay=0, nesterov=True)
# scaler = amp.GradScaler()
for epoch in range(1): #625
    # train_ls = strain(labeled_loader, model, loss_fn, optimizer, scheduler)
    train_ls = strain(labeled_loader, model, criterion, optimizer)
    evaluate(test_loader, student_model, criterion)


In [15]:
correct, test_loss = evaluate(test_loader, student_model, criterion)
print(correct, test_loss)

BackendCompilerFailed: ignored

In [16]:
# @title save

from google.colab import drive
drive.mount('/content/drive')


checkpoint = {
'epoch': t+1,
'teacher_model': teacher_model.state_dict(),
'student_model': student_model.state_dict(),
'avg_student_model': avg_student_model.state_dict(),
't_optimizer': t_optimizer.state_dict(),
's_optimizer': s_optimizer.state_dict(),
't_scheduler': t_scheduler.state_dict(),
's_scheduler': s_scheduler.state_dict(),}
# torch.save(checkpoint, pth)
torch.save(checkpoint, 'ckpt.pth')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
